In [2]:
!pip install opencv-python
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [1]:
categories = os.listdir("C:/Users/Shireen/A FINAL SER ATTEMPT/data/spectograms/images/")
def load_images_and_labels(categories):
    img_lst=[]
    labels=[]
    for index, category in enumerate(categories):
        for image_name in os.listdir(fpath+"/"+category):
            img = cv2.imread(fpath+"/"+category+"/"+image_name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            img_array = Image.fromarray(img, 'RGB')
            
            #resize image to 224 x 224 because the input image resolution for Resnet50 is 224 x 224
            resized_img = img_array.resize((224, 224))
            
            img_lst.append(np.array(resized_img))
            
            labels.append(index)
    return img_lst, labels

fpath = "C:/Users/Shireen/A FINAL SER ATTEMPT/data/spectograms/images/"
images, labels = load_images_and_labels(categories)
print("No. of images loaded = ",len(images),"\nNo. of labels loaded = ",len(labels))
print(type(images),type(labels))

NameError: name 'os' is not defined

In [3]:
images = np.array(images)
labels = np.array(labels)

print("Images shape = ",images.shape,"\nLabels shape = ",labels.shape)
print(type(images),type(labels))

Images shape =  (1440, 224, 224, 3) 
Labels shape =  (1440,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [4]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

print("x_train shape = ",x_train.shape)
print("y_train shape = ",y_train.shape)
print("\nx_test shape = ",x_test.shape)
print("y_test shape = ",y_test.shape)

x_train shape =  (1152, 224, 224, 3)
y_train shape =  (1152,)

x_test shape =  (288, 224, 224, 3)
y_test shape =  (288,)


In [5]:
from keras.utils import np_utils
from keras.utils import to_categorical
X_train = x_train/255.
X_test = x_test/255.

# Convert training and test labels to one hot matrices
Y_train = np_utils.to_categorical(y_train, 8)
Y_test = np_utils.to_categorical(y_test, 8)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1152
number of test examples = 288
X_train shape: (1152, 224, 224, 3)
Y_train shape: (1152, 8)
X_test shape: (288, 224, 224, 3)
Y_test shape: (288, 8)


In [6]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

In [7]:
img_height,img_width = 224,224 
num_classes = 8
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = ResNet50(weights= 'imagenet', include_top=False, input_shape= (224,224,3))

In [8]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
from keras.layers import GlobalAveragePooling2D

In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [11]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(X_train, Y_train, epochs = 20, batch_size=64, validation_data=(X_test, Y_test))

Epoch 1/20
18/18 [==============================] - 748s 42s/step - loss: 2.3827 - accuracy: 0.2179 - val_loss: 2.3746 - val_accuracy: 0.1215
Epoch 2/20
18/18 [==============================] - 826s 46s/step - loss: 1.2610 - accuracy: 0.5530 - val_loss: 2.3425 - val_accuracy: 0.1215
Epoch 3/20
18/18 [==============================] - 782s 43s/step - loss: 0.6485 - accuracy: 0.7752 - val_loss: 2.3189 - val_accuracy: 0.1215
Epoch 4/20
18/18 [==============================] - 997s 55s/step - loss: 0.3123 - accuracy: 0.9123 - val_loss: 2.3146 - val_accuracy: 0.1458
Epoch 5/20
18/18 [==============================] - 1175s 65s/step - loss: 0.1559 - accuracy: 0.9644 - val_loss: 2.2359 - val_accuracy: 0.1111
Epoch 6/20
18/18 [==============================] - 1138s 63s/step - loss: 0.0752 - accuracy: 0.9844 - val_loss: 2.2409 - val_accuracy: 0.1458
Epoch 7/20
18/18 [==============================] - 1218s 68s/step - loss: 0.0387 - accuracy: 0.9965 - val_loss: 2.2731 - val_accuracy: 0.1458
Epo

In [13]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.2395379543304443
Test accuracy: 0.1597222238779068
